In [ ]:
%%capture
!pip install langchain openai datasets faiss-gpu sentence_transformers

In [ ]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter Your OpenAI API Key:")

Enter Your OpenAI API Key:··········


## Self-Consistency Prompting in Language Models

Self-consistency prompting was introduced in the March 2022 paper ["Self-Consistency Improves Chain of Thought Reasoning in Language Models"](https://arxiv.org/pdf/2203.11171.pdf) by Xuezhi Wang, et. al.

# 🤔 **What is Self-Consistency Prompting?**

- Focuses on exploring different reasoning paths for complex problems.

- Aims for reliable answers by checking consistency across various thought processes.

### 🌟 **Differences from Other Prompting Techniques**

- Traditional CoT: Generates short sentences mimicking human reasoning for solving tasks.

- Self-Consistency: Samples multiple reasoning paths and finds the most consistent answer.

- It's unsupervised and works with pre-trained models, unlike methods needing extra training or human annotations.

### **🛠️ Constructing a Self-Consistency Prompt**


<figure>
  <img src="https://ar5iv.labs.arxiv.org/html/2203.11171/assets/x1.png" alt="Image Description" style="width:100%">
  <figcaption>The self-consistency method contains three steps: (1) prompt a language model using chain-of-thought (CoT) prompting; (2) replace the “greedy decode” in CoT prompting by sampling from the language model’s decoder to generate a diverse set of reasoning paths; and (3) marginalize out the reasoning paths and aggregate by choosing the most consistent answer in the final answer set.</figcaption>
  <a href="https://ar5iv.labs.arxiv.org/html/2203.11171">Image Source</a>
</figure>


1. **Start with CoT**: Use chain-of-thought prompting as the base.

2. **Sample Diverse Paths**: Use "sample-and-marginalize" decoding to get various reasoning paths.

3. **Marginalize for Consistency**: Find the most consistent answer from these different paths.

### 🔍 **Examples in Action**

- **Chain-of-Thought**: For the question "If there are 3 cars in the parking lot and 2 more cars arrive, how many cars are in the parking lot?", instead of directly answering "5", the model might respond with the reasoning: "There are 3 cars in the parking lot already. 2 more arrive. Now there are 3 + 2 = 5 cars. The answer is 5."

- **Self-Consistency**: For a question about how much Janet makes from selling eggs, the model might generate multiple reasoning paths like:
  1. "She has 16 - 3 - 4 = 9 eggs left. So she makes $2 * 9 = $18 per day."

  2. "This means she sells the remainder for $2 * (16 - 4 - 3) = $26 per day."
  
  3. "She eats 3 for breakfast, so she has 16 - 3 = 13 left. Then she bakes muffins, so she has 13 - 4 = 9 eggs left. So she has 9 eggs * $2 = $18."

  The model then aggregates these paths to determine the most consistent answer, which in this case is "$18 per day."


🚀 **Impact of Self-Consistency Prompting**
- Enhances model reasoning by considering multiple paths.
- Shown to boost performance in arithmetic and commonsense reasoning tasks.

# Begin by setting up CoT prompts

We're following the same pattern from the Chain of Thought lesson.

1. Downloading CoT prompt datasets from HuggingFace

2. Downloading embeddings model from HuggingFace

3. Creating prompt template for CoT

4. Creating an example selector

5. Construct the prompt

In [ ]:
%%capture
from datasets import load_dataset

dataset = load_dataset("kaist-ai/CoT-Collection")

dataset['train']

examples = dataset['train'].to_pandas()[['source', 'target', 'rationale']]

selected_examples = examples.sample(n=10_000).to_dict(orient='records')

In [ ]:
%%capture
from langchain.embeddings import HuggingFaceBgeEmbeddings

model_name = "BAAI/bge-base-en-v1.5"

encode_kwargs = {'normalize_embeddings': True} # set True to compute cosine similarity

embeddings = HuggingFaceBgeEmbeddings(
    model_name=model_name,
    model_kwargs={'device': 'cuda'},
    encode_kwargs=encode_kwargs
)

In [ ]:
from langchain.prompts.example_selector import MaxMarginalRelevanceExampleSelector
from langchain.vectorstores import FAISS
from langchain.prompts import FewShotPromptTemplate, PromptTemplate

prefix = "Consider the following as examples of how to reason:"

examples_template = """Query: {source}

Rationale: {rationale}

Response: {target}
"""

suffix = """Using a similar reasoning approach, answer the users question which is delimited by triple backticks.

User question: ```{input}```

Take a deep breath, break down the user's query step-by-step, and provide a clear chain of thought in your response."
"""

examples_prompt = PromptTemplate(
    input_variables=["source", "rationale", "target"],
    template=examples_template
)

In [ ]:
example_selector = MaxMarginalRelevanceExampleSelector.from_examples(
    selected_examples,
    embeddings,
    FAISS,
    k=5,
)

mmr_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=examples_prompt,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input"]
)

In [ ]:
query = """There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
"""
prompt = mmr_prompt.format(input=query)

In [ ]:
print(prompt)

Consider the following as examples of how to reason:

Query: Where would a computer user be using their own computer?

[Options]
- hell
- school
- indoors
- internet cafe
- house

Rationale: A computer user would be using their own computer in their house because computer users use their personal computers and other personal belongings in their house.

Response: house


Query: Ali has a small flower shop. He sold 4 flowers on Monday, 8 flowers on Tuesday and on Friday, he sold double the number of flowers he sold on Monday. How many flowers does Ali sell?

Rationale: Combining Tuesday and Monday, Ali sold 4 + 8 = 12 flowers. On Friday, he sold 2 * 4 = 8 flowers. Altogether, Ali sold 12 + 8 = 20 flowers.

Response: 20


Query: Article: Construction of the chapel, originally intended to be slightly over twice as long, with eighteen - or possibly seventeen - bays (there are eight today) was stopped when Henry VI was deposed. Only the Quire of the intended building was completed. Eton's fi

# Self-Consistency Prompt

In [ ]:
sc_template = """Based on the responses (delimited by < >) to the following query, \
(delimited by triple backticks) return the response that occurs most frequently.

Query: ```{query}```

Responses: <{responses}>
"""

sc_prompt = PromptTemplate.from_template(sc_template)

### Generating multiple responses


Use the `n` parameter to generate alternative responses. Increase `n` to explore different variations.



In [ ]:
from langchain.llms import OpenAI

llm = OpenAI(n=5)

generations = llm.generate([prompt])

In [ ]:
generations.generations

[[Generation(text="\nRationale: The user's query states that there were nine computers in the server room initially and that five more computers were installed each day from Monday to Thursday. Therefore, the number of computers in the server room on Thursday is 9 + 5 + 5 + 5 + 5 = 29.\n\nResponse: 29", generation_info={'finish_reason': 'stop', 'logprobs': None}),
  Generation(text='\nRationale: At the start of the week, there were nine computers in the server room. On Monday, five more computers were added to the server room, making the total number of computers 14. Similarly, on Tuesday, Wednesday and Thursday, five more computers were added each day, making the total number of computers now in the server room 19.\n\nResponse: 19', generation_info={'finish_reason': 'stop', 'logprobs': None}),
  Generation(text='\nRationale: There were nine computers in the server room at the beginning. Then, five more computers were installed each day from Monday to Thursday. So, Monday would have 5 

In [ ]:
responses = []
for item in generations.generations[0]:
    response_index = item.text.find("Response: ")
    if response_index != -1:
        response = item.text[response_index + len("Response: "):].strip()
        responses.append(response)

In [ ]:
responses

['29', '19', '59', '24', '29']

In [ ]:
llm = OpenAI()

final_prompt = sc_prompt.format(query=query, responses=str(responses))

print(final_prompt)

Based on the responses (delimited by < >) to the following query, (delimited by triple backticks) return the response that occurs most frequently.

Query: ```There were nine computers in the server room. Five more computers were installed each day, from
monday to thursday. How many computers are now in the server room?
```

Responses: <['29', '19', '59', '24', '29']>



In [ ]:
print(llm(final_prompt))


29
